# Healthcare Insurance Personal Prediction

This is a machine learning project to predict healthcare insurance charges for individuals based on various features such as age, gender, BMI, smoking habit, and region. The goal is to build a regression model that can accurately predict the insurance charges for new individuals based on their features.

In [16]:
# Import Libraries
import time
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def adjusted_r2_score(y_test, y_pred, n, p):
    score = r2_score(y_test, y_pred)
    return 1 - (1 - score) * (n - 1) / (n - p - 1)

import matplotlib
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import HTML

def multi_table(table_list):
    return HTML(
        f"<table><tr> {''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list])} </tr></table>")

We load the insurance dataset and explore its features and target variable. We also check for missing values and outliers.

In [17]:
df_insurance = pd.read_csv("insurance.csv")
display(df_insurance)
print(df_insurance.info())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None


In [18]:
multi_table([pd.DataFrame(df_insurance[col].value_counts()) for col in df_insurance.columns])

,age
18,69
19,68
50,29
51,29
47,29
46,29
45,29
20,29
48,29
52,29


We transform the categorical features into numerical ones using one-hot encoding. We also normalize the numerical features to have zero mean and unit variance.

In [19]:
print("Before Categorical Encoding:", df_insurance.shape)

# Binary
df_insurance["sex"] = df_insurance["sex"].replace(["male", "female"], [0, 1])
df_insurance["smoker"] = df_insurance["smoker"].replace(["no", "yes"], [0, 1])

# Categorical
categorical_features = ["region"]
df_category = pd.get_dummies(data=df_insurance[categorical_features], drop_first=True)
df_insurance = pd.concat([df_insurance, df_category], axis=1)
df_insurance = df_insurance.drop(categorical_features, axis=1)

print(" After Categorical Encoding:", df_insurance.shape)

display(df_insurance)

features = df_insurance.drop("charges", axis=1)
labels = df_insurance["charges"]

Before Categorical Encoding: (1338, 7)
 After Categorical Encoding: (1338, 9)


,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,19,1,27.900,0,1,16884.92400,0,0,1
1,18,0,33.770,1,0,1725.55230,0,1,0
2,28,0,33.000,3,0,4449.46200,0,1,0
3,33,0,22.705,0,0,21984.47061,1,0,0
4,32,0,28.880,0,0,3866.85520,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,0,30.970,3,0,10600.54830,1,0,0
1334,18,1,31.920,0,0,2205.98080,0,0,0
1335,18,1,36.850,0,0,1629.83350,0,1,0
1336,21,1,25.800,0,0,2007.94500,0,0,1


# Prediction
Model Training and Inference
We train and evaluate several regression models such as Linear Regression, Ridge Regression, Lasso Regression, Decision Tree Regression, Random Forest Regression, Gradient Boosting Regression, and Support Vector Regression. We use k-fold cross-validation to select the best model based on its R2 score.

# Hyperparameter Tuning
We perform hyperparameter tuning for the best models to further improve their performance.


In [20]:
# Models and Parameters

# Linear Regression
lr_model = LinearRegression()
linear_params = {'model__alpha': np.linspace(0, 1.0, 20)} 

# Ridge
ridge_model = Ridge()

# Lasso
lasso_model = Lasso()

# ElasticNet
elnet_model = ElasticNet()

# KNN
knn_model = KNeighborsRegressor()
knn_params = {'model__n_neighbors': list(range(1, 21))}

# SVM
svm_model = SVR()
svm_params = {
    'model__kernel': ['linear', 'rbf', 'poly'],
    'model__C': [1, 10, 50, 100, 200, 500]
}

# Decision Tree
dt_model = DecisionTreeRegressor(random_state=2023)
dt_params = {
    'model__max_features': ['auto', 'log2'],
    'model__ccp_alpha': [0.1, 0.01],
    'model__max_depth': [4, 5, 6, 7, 8]
}

# Random Forest
rf_model = RandomForestRegressor(random_state=2023)
rf_params = {
    'model__n_estimators': [250, 500],
    'model__max_features': ['auto', 'log2'],
    'model__max_depth' : [4, 5, 6, 7, 8],
}

models = [lr_model, ridge_model, lasso_model, elnet_model, lr_model, knn_model, svm_model, dt_model, rf_model]
params = [None, linear_params, linear_params, linear_params, None, knn_params, svm_params, dt_params, rf_params]
model_names = ["Linear Regression", "Ridge", "Lasso", "Elastic Net", "Polynomial Regression", "K-Nearest Neighbors", "Support Vector Machine", "Decision Tree", "Random Forest"]

In [21]:
best_score = []
best_params = []
for i in range(9):
    start_time = time.time()
    kf = KFold(n_splits=5, shuffle=True, random_state=2023)
    print(model_names[i])
    
    # Check Params
    if(params[i]==None):
        if(model_names[i]=="Polynomial Regression"):
            pipe = Pipeline([
                ('poly', PolynomialFeatures(degree=2)),
                ('scaler', MinMaxScaler()),
                ('model', models[i]),
            ])
            scores = cross_val_score(pipe, features, labels, scoring='r2', cv=kf)
        else:
            pipe = Pipeline([
                ('scaler', MinMaxScaler()),
                ('model', models[i]),
            ])
            scores = cross_val_score(pipe, features, labels, scoring='r2', cv=kf)
        mean_scores = np.mean(scores)
        print("Best Score:", mean_scores)
        best_score.append(mean_scores)
        best_params.append(None)
    else:
        pipe = Pipeline([
            ('scaler', MinMaxScaler()),
            ('model', models[i]),
        ])
        clf = GridSearchCV(pipe, params[i], scoring="r2", cv=kf, verbose=1)
        clf.fit(features, labels)

        print("Best Score:", clf.best_score_)
        print("Best Params:", clf.best_params_)
        best_score.append(clf.best_score_)
        best_params.append(clf.best_params_)
    print("--- %s seconds ---" % (time.time() - start_time))
    print()

Linear Regression
Best Score: 0.7479038843395849
--- 0.03590083122253418 seconds ---

Ridge
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score: 0.7479176867903254
Best Params: {'model__alpha': 0.47368421052631576}
--- 0.605482816696167 seconds ---

Lasso
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score: 0.747923753947914
Best Params: {'model__alpha': 1.0}
--- 0.6539368629455566 seconds ---

Elastic Net
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score: 0.7479038843395849
Best Params: {'model__alpha': 0.0}
--- 0.7104103565216064 seconds ---

Polynomial Regression
Best Score: 0.8383982574628993
--- 0.14609313011169434 seconds ---

K-Nearest Neighbors
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score: 0.7689407837689035
Best Params: {'model__n_neighbors': 5}
--- 0.9285855293273926 seconds ---

Support Vector Machine
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Score: 0.80264662

# Best Models
We select the best models based on their R2 score and evaluate their performance on the test set.


In [22]:
# Create Data
df_r2_scores = pd.DataFrame({'Model': model_names, 'Best Params': best_params, "R2 Score": best_score})
df_r2_scores = df_r2_scores.sort_values(by="R2 Score", ascending=True).reset_index(drop=True)
df_r2_scores["R2 Score"] = df_r2_scores["R2 Score"] * 100
df_r2_scores

,Model,Best Params,R2 Score
0,Linear Regression,None,74.790388
1,Elastic Net,{'model__alpha': 0.0},74.790388
2,Ridge,{'model__alpha': 0.47368421052631576},74.791769
3,Lasso,{'model__alpha': 1.0},74.792375
4,K-Nearest Neighbors,{'model__n_neighbors': 5},76.894078
5,Support Vector Machine,"{'model__C': 500, 'model__kernel': 'poly'}",80.264663
6,Polynomial Regression,None,83.839826
7,Decision Tree,"{'model__ccp_alpha': 0.1, 'model__max_depth': ...",85.172789
8,Random Forest,"{'model__max_depth': 4, 'model__max_features':...",86.055602


In [23]:
best_models = df_r2_scores.iloc[-1, :].copy()
final_params = best_models["Best Params"]
list_keys = list(final_params)
for key in list_keys:
    string = key[7:]
    final_params[string] = final_params[key]
    del final_params[key]
print(final_params)

y_result = pd.DataFrame()
feature_importances = []
    
kf = KFold(n_splits=5, shuffle=True, random_state=2023)
for fold, (train_index, test_index) in enumerate(kf.split(features)):
    print("Fold {} :".format(fold))
    X_train, X_test = features.loc[train_index, :], features.loc[test_index, :]
    y_train, y_test = labels.loc[train_index], labels.loc[test_index]
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    n = X_test.shape[0]
    p = X_test.shape[1]
    
    model = RandomForestRegressor(random_state=2023)
    model.set_params(**final_params)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    df_fold = pd.DataFrame({"Index": test_index, "Actual": y_test, "Predicted": y_pred, "Fold": [fold] * n})
    print(df_fold.shape)
    
    y_result = y_result.append(df_fold, ignore_index=True)
    feature_importances.append(model.feature_importances_)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    adj_r2 = adjusted_r2_score(y_test, y_pred, n, p)
    
    print("MSE    :", mse)
    print("RMSE   :", rmse)
    print("MAE    :", mae)
    print("R2     :", r2)
    print("Adj R2 :", adj_r2)
    print()

y_result = y_result.sort_values(by="Index", ascending=True).reset_index(drop=True)
y_result = y_result.drop("Index", axis=1)

final_mse = mean_squared_error(y_result["Actual"], y_result["Predicted"])
final_rmse = mean_squared_error(y_result["Actual"], y_result["Predicted"], squared=False)
final_mae = mean_absolute_error(y_result["Actual"], y_result["Predicted"])
final_r2 = r2_score(y_result["Actual"], y_result["Predicted"])
final_adj_r2 = adjusted_r2_score(y_result["Actual"], y_result["Predicted"], y_result.shape[0], p)

print("Final Score")
print("MSE    :", final_mse)
print("RMSE   :", final_rmse)
print("MAE    :", final_mae)
print("R2     :", final_r2)
print("Adj R2 :", final_adj_r2)

{'max_depth': 4, 'max_features': 'auto', 'n_estimators': 500}
Fold 0 :
(268, 4)
MSE    : 19636083.74546704
RMSE   : 4431.262093971315
MAE    : 2629.6057319621286
R2     : 0.8508744522851982
Adj R2 : 0.8462682577611889

Fold 1 :
(268, 4)
MSE    : 25061705.500318047
RMSE   : 5006.166747154758
MAE    : 2669.4145013572397
R2     : 0.8382391201394602
Adj R2 : 0.8332426450858528

Fold 2 :
(268, 4)
MSE    : 24435133.81310222
RMSE   : 4943.190651097954
MAE    : 2769.2048742895167
R2     : 0.843102095040575
Adj R2 : 0.8382558277059209

Fold 3 :
(267, 4)
MSE    : 20565093.86819458
RMSE   : 4534.875286950522
MAE    : 2525.966819187983
R2     : 0.8623110798138095
Adj R2 : 0.8580416559320672

Fold 4 :
(267, 4)
MSE    : 21995943.37322994
RMSE   : 4689.983302020375
MAE    : 2650.833900549691
R2     : 0.8428268670105413
Adj R2 : 0.8379532814914883

Final Score
MSE    : 22340373.933663785
RMSE   : 4726.560476040033
MAE    : 2649.0957556122125
R2     : 0.8475504832601383
Adj R2 : 0.8466328037011324



# Visualizations
We create several visualizations to better understand the models and their predictions.


In [24]:
# Create Figure
fig = go.Figure()

# Color
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#d60041","#d60061", "#b52182", "#5533a2"])
min_color = df_r2_scores["R2 Score"].min()
max_color = df_r2_scores["R2 Score"].max()
colors = []
for i in range(9):
    color = cmap(i/8)
    color = matplotlib.colors.rgb2hex(color)
    colors.append(color)

# Lollipop Chart
fig.add_trace(
    go.Scatter(
        x=df_r2_scores["R2 Score"],
        y=df_r2_scores["Model"],
        mode='markers+text',
        marker=dict(
             color=colors,
             size=50,
        ),
        text=["{:.2f}%".format(x) for x in df_r2_scores["R2 Score"]],
        textposition="middle center",
        textfont=dict(
            size=13,
            color="White"
        ),
    )
)

for i in range(9):
    fig.add_shape(type="line",
                  x0=0.0, y0=i, x1=df_r2_scores["R2 Score"][i]-0.65, y1=i,
                  line=dict(
                      color=colors[i],
                      width=6.5
                  )
     )

# Update Axes
fig.update_xaxes(title_text="", showticklabels=False, showgrid=False, range=[70.0, 87.0])
fig.update_yaxes(title_text="", showticklabels=True, showgrid=False)

# Update Layout
fig.update_layout(title_text='Comparison Models', title_x=0.5, font_family="Calibri", font_size=15,
                  margin=dict(
                      pad=20
                  ),
                  width=900, height=800,
                  plot_bgcolor='White',
                  showlegend=False,
)

# Show
fig.show(renderer="iframe_connected")


# Comparison Models with R2 Score
We compare the R2 score of the best models and visualize their performance using a bar chart.


Feature Importance
We visualize the feature importance of the best models to understand which features are most important in predicting the insurance charges.

In [25]:

# Create DataFrame
df_importance = pd.DataFrame()
df_importance["Features"] = features.columns
df_importance["Importance"] = np.mean(feature_importances, axis=0)
df_importance = df_importance.sort_values(by="Importance", ascending=True).reset_index(drop=True)

# Create Figure
fig = go.Figure()

# Color
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#d60041","#d60061", "#b52182", "#5533a2"])
min_color = df_importance["Importance"].min()
max_color = df_importance["Importance"].max()
colors = []
for i in range(8):
    color = cmap(i/7)
    color = matplotlib.colors.rgb2hex(color)
    colors.append(color)

# Bar Chart
fig.add_trace(
    go.Bar(
        x=df_importance["Importance"],
        y=[1, 2, 3, 4, 5, 6, 7, 8],
        orientation='h',
        marker_color=colors,
        name="Feature Importance"
    )
)

fig.add_trace(
    go.Bar(
        x=[1.0] * 8,
        y=[1, 2, 3, 4, 5, 6, 7, 8],
        orientation='h',
        marker_color="#e6e6e6",
        hoverinfo='skip',
    )
)

# Update Axes
fig.update_xaxes(title_text="", range=[0, 1.0], showticklabels=True, showgrid=False, ticks="outside", linecolor="Black")
fig.update_yaxes(title_text="", showgrid=False, 
             tickmode='array', tickvals=[1, 2, 3, 4, 5, 6, 7, 8], ticktext=df_importance["Features"])

# Update Layout
fig.update_layout(
    title="Feature Importance", title_x=0.5, font_family="Calibri", font_size=14,
    width=900, height=450,
    showlegend=False,
    plot_bgcolor="White",
    barmode="stack", bargap=0.45,
    margin=dict(
        pad=10
    )
)

# Show
fig.show(renderer="iframe_connected")

# Linearity (Actual vs Predicted Values)
We create scatter plots to compare the actual and predicted insurance charges for the best models and check for linearity.


In [26]:
# Create Figure
fig = go.Figure()

# Scatter Plot
fig.add_trace(
    go.Scatter(
        x=y_result["Actual"],
        y=y_result["Predicted"],
        mode="markers",
        marker=dict(
            color="#d60041",
            size=5
        ),
        hoverinfo="skip"
    )
)

fig.add_trace(
    go.Scatter(
        x=[y_result["Actual"].min(), y_result["Actual"].max()],
        y=[y_result["Actual"].min(), y_result["Actual"].max()],
        mode="lines",
        line=dict(
            color="#5533a2",
            width=3
        ),
        hoverinfo="skip"
    )
)

# Update Axes
fig.update_xaxes(title="Actual Values", ticks="outside", linecolor="Black")
fig.update_yaxes(title="Predicted Values", ticks="outside", linecolor="Black")

# Update Layout
fig.update_layout(
    title="Linearity (Actual vs Predicted Values)", title_x=0.5, font_family="Calibri",
    width=600, height=600,
    plot_bgcolor="White",
    showlegend=False
)

# Show
fig.show(renderer="iframe_connected")

# Residual Plot
We create residual plots to check for homoscedasticity and normality of the errors.

In [27]:
y_result["Residual"] = y_result["Actual"] - y_result["Predicted"]

# Create Figure
fig = go.Figure()

# Scatter Plot
fig.add_trace(
    go.Scatter(
        x=y_result["Actual"],
        y=y_result["Residual"],
        mode="markers",
        marker=dict(
            color="#d60041",
            size=5
        ),
        hoverinfo="skip"
    )
)

fig.add_trace(
    go.Scatter(
        x=[y_result["Actual"].min(), y_result["Actual"].max()],
        y=[0.0, 0.0],
        mode="lines",
        line=dict(
            color="#5533a2",
            width=3
        ),
        hoverinfo="skip"
    )
)

# Update Axes
fig.update_xaxes(title="Charges", ticks="outside", linecolor="Black")
fig.update_yaxes(title="Residual", ticks="outside", linecolor="Black")

# Update Layout
fig.update_layout(
    title="Residual Plots", title_x=0.5, font_family="Calibri",
    width=600, height=600,
    plot_bgcolor="White",
    showlegend=False
)

# Show
fig.show(renderer="iframe_connected")